In [77]:
using JuMP
using Ipopt
using MadNLP
using Complementarity

In [49]:
GU = Dict(:pi => 0.01, 
          :L => 0.5, 
          :sigma => 0.9, 
          :kvalue => 0, 
          :nfirm => 1, 
          :rho => 0, 
          :gammamax => 0)

GU[:rho] = 1-1/GU[:sigma]
GU[:gammamax] = GU[:pi]*(1-GU[:L])^(-1/GU[:sigma]) / (GU[:pi] * (1-GU[:L])^(-1/GU[:sigma]) + 1 - GU[:pi])

0.021353469024510078

In [148]:
function insurance(GU)
    pi = GU[:pi]
    L = GU[:L]
    σ = GU[:sigma]
    kvalue = GU[:kvalue]
    nfirm = GU[:nfirm]
    ρ = GU[:rho]
    gammamax = GU[:gammamax]


    m = MCPModel()#Model(Ipopt.Optimizer)

    @variables(m,begin
         P_G>=1e-5, (start = 1-pi,)	#Price index for consumption on a good day,
		 P_B>=1e-5, (start = pi,)	#Price index for consumption on a bad day,
		 C_G, (start = 1-pi + pi*(1-L),)	#Consumption in the good day,
		 C_B, (start = 1-pi + pi*(1-L),)	#Consumption in the bad day,
		 P_C>=1e-5, (start = 1,)	#Consumption price index,
		 M, (start = 1-pi + pi*(1-L),)	#Income,
		 K>=0, (start = (1-pi + pi*(1-L) - (1-L))/(1-pi),)	#Coverage
		GAMMA>=0, (start = pi,)	#Premium for coverage
		 PROFIT	#Firm profit
    end)

    fix(P_G,1-pi,force=true)
    fix(GAMMA,pi,force=true)

    @mapping(m,income, M - P_G - P_B*(1-L))
    @mapping(m,P_Cdef, P_C - (( (1-pi) * (P_G/(1-pi))^(1-σ) + pi * (P_B/pi)^(1-σ) )^(1/(1-σ))))
    @mapping(m,C_Gdef, C_G - (M/P_C * (P_C*(1-pi)/P_G)^σ))
    @mapping(m,C_Bdef, C_B - (M/P_C * (P_C*pi/P_B)^σ))
    @mapping(m,market_G, 1 - C_G - GAMMA*K)
    @mapping(m,market_B, 1 - L + K - C_B - GAMMA*K)
    @mapping(m,demand_K, GAMMA*(P_G+P_B) - P_B)
    @mapping(m,profitdef, PROFIT - (GAMMA-pi)*(K-kvalue*(nfirm-1)/nfirm))
    @mapping(m,dummy,0)
    

    @complementarity(m,income,M)
    @complementarity(m,P_Cdef,P_C)
    @complementarity(m,market_G,P_G) 
    @complementarity(m,market_B,P_B) 
    @complementarity(m,C_Gdef,C_G)
    @complementarity(m,C_Bdef,C_B) 
    @complementarity(m,demand_K,K) 
    @complementarity(m,profitdef,PROFIT)
    @complementarity(m,dummy,GAMMA)

    return m
end


function nash(GU)
    pi = GU[:pi]
    L = GU[:L]
    σ = GU[:sigma]
    kvalue = GU[:kvalue]
    nfirm = GU[:nfirm]
    ρ = GU[:rho]
    gammamax = GU[:gammamax]


    m = Model(MadNLP.Optimizer)

    @variables(m,begin
         P_G>=1e-5, (start = 1-pi,)	#Price index for consumption on a good day,
		 P_B>=1e-5, (start = pi,)	#Price index for consumption on a bad day,
		 C_G, (start = 1-pi + pi*(1-L),)	#Consumption in the good day,
		 C_B, (start = 1-pi + pi*(1-L),)	#Consumption in the bad day,
		 P_C>=1e-5, (start = 1,)	#Consumption price index,
		 M, (start = 1-pi + pi*(1-L),)	#Income,
		 K>=0, (start = (1-pi + pi*(1-L) - (1-L))/((1-pi)),)	#Coverage
		0<=GAMMA<=gammamax, (start = gammamax/2,)	#Premium for coverage
    end)

    fix(P_G,1-pi,force=true)

    @NLobjective(m,Min,-(GAMMA-pi)*(K-kvalue*(nfirm-1)/nfirm))

    @NLconstraints(m,begin
        income, M == P_G + P_B*(1-L)
        P_Cdef, P_C == ( (1-pi) * (P_G/(1-pi))^(1-σ) + pi * (P_B/pi)^(1-σ) )^(1/(1-σ))
        C_Gdef, C_G == M/P_C * (P_C*(1-pi)/P_G)^σ
        C_Bdef, C_B == M/P_C * (P_C*pi/P_B)^σ
        market_G, 1 == C_G + GAMMA*K
        market_B, 1 - L + K == C_B + GAMMA*K
        demand_K, GAMMA*(P_G+P_B) == P_B
    end)


    return m
end

nash (generic function with 1 method)

In [155]:
GU[:nfirm] = 2
GU[:kvalue] = .5

Nash_ipopt = nash(GU)
optimize!(Nash_ipopt)

for var in all_variables(Nash_ipopt)
    println("$var -> $(value(var))")
end

objective_value(Nash_ipopt)

This is MadNLP version v0.6.0, running with umfpack

Number of nonzeros in constraint Jacobian............:       23
Number of nonzeros in Lagrangian Hessian.............:       30

Total number of variables............................:        7
                     variables with only lower bounds:        3
                variables with lower and upper bounds:        1
                     variables with only upper bounds:        0
Total number of equality constraints.................:        7
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 -1.6918363e-04 8.94e-04 1.00e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00   0
   1  2.3898347e-08 8.65e-07 1.64e-01  -1.0 6.77e-04    -  1.00e+00 1.00e+00h  

-0.00019971826862568071

In [152]:
GU[:kvalue] = (1-GU[:pi] + GU[:pi]*(1-GU[:L]) - (1-GU[:L]))/(1-GU[:pi])

log = Dict()

for n=1:15
    dev = 1
    GU[:nfirm] = n
    for iter∈1:25
        if dev ≈ 0 
            break
        end
        Nash = nash(GU)
        set_silent(Nash)
        optimize!(Nash)

        log[n,iter] = Dict(
            :dev => dev,
            :K => value(Nash[:K]),
            :Gamma => value(Nash[:GAMMA]),
            :Profit => (value(Nash[:GAMMA]) - GU[:pi])/value(Nash[:K]),
            :P_B_pi => value(Nash[:P_B])/GU[:pi],
            :obj => objective_value(Nash)
        )

        dev = abs(GU[:kvalue] - value(Nash[:K]))
        GU[:kvalue] = value(Nash[:K])
    end
end


In [153]:
out = "          K    Gamma   Profit   P_B/pi   dev\n"       

for key in sort(collect(keys(log)))
    x = log[key]
    K = round(x[:K],digits=3)
    G = round(x[:Gamma],digits=3)
    P = round(x[:Profit],digits=3)
    P_B = round(x[:P_B_pi],digits=3)
    dev = round(x[:dev],digits=3)
    obj = x[:obj]
    out*="$key   $K   $G   $P   $P_B    $dev    $obj\n"
end
print(out)

          K    Gamma   Profit   P_B/pi   dev
(1, 1)   0.346   0.012   0.006   1.206    1.0    -0.0007027353964662583
(1, 2)   0.346   0.012   0.006   1.206    0.154    -0.0007029057165092508
(1, 3)   0.346   0.012   0.006   1.206    0.0    -0.0007027353964662583
(1, 4)   0.346   0.012   0.006   1.206    0.0    -0.0007029057165092508
(1, 5)   0.346   0.012   0.006   1.206    0.0    -0.0007027353964662583
(1, 6)   0.346   0.012   0.006   1.206    0.0    -0.0007029057165092508
(1, 7)   0.346   0.012   0.006   1.206    0.0    -0.0007029057165092508
(2, 1)   0.395   0.011   0.003   1.132    1.0    -0.00029025369823422817
(2, 2)   0.334   0.012   0.007   1.224    0.049    -0.0003031916710044911
(2, 3)   0.317   0.012   0.008   1.253    0.06    -0.00037421584870498564
(2, 4)   0.344   0.012   0.006   1.208    0.017    -0.0003817348421950489
(2, 5)   0.471   0.01   0.001   1.034    0.027    -9.976639212320692e-5
(2, 6)   0.419   0.011   0.002   1.099    0.127    -0.00018023771763007256
(2, 7) 

In [140]:
using TypedTables

T = Table(a=[1],b=[2])
T[2] = (a=1,b=2)

BoundsError: BoundsError: attempt to access 1-element Table{NamedTuple{(:a, :b), Tuple{Int64, Int64}}, 1, NamedTuple{(:a, :b), Tuple{Vector{Int64}, Vector{Int64}}}} at index [2]

In [119]:
L = [round(b[:K],digits=8) for (a,b) in log]
unique(L)


351-element Vector{Float64}:
 0.47644567
 0.47425157
 0.44476691
 0.43978977
 0.49920944
 0.44004972
 0.47580019
 0.49789895
 0.47458422
 0.4827861
 ⋮
 0.47702263
 0.47402547
 0.42101847
 0.43026993
 0.41367428
 0.45031724
 0.43413068
 0.48580546
 0.47380068

In [53]:
log[1,1]

Dict{Symbol, Real} with 5 entries:
  :Gamma  => 0.0099999
  :K      => 0.500009
  :Profit => -1.96777e-7
  :dev    => 1
  :P_B_pi => 0.99999

In [61]:
GU

Dict{Symbol, Real} with 7 entries:
  :rho      => -0.111111
  :L        => 0.5
  :gammamax => 0.0213535
  :sigma    => 0.9
  :nfirm    => 2
  :pi       => 0.01
  :kvalue   => 0.5

2
This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:       19
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       22

Total number of variables............................:        7
                     variables with only lower bounds:        3
                variables with lower and upper bounds:        1
                     variables with only upper bounds:        0
Total number of equality constraints.................:        7
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.6918363e-04 8.94e-04 1.00e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00